In [462]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [463]:
df = pd.read_excel('AtlasBrasil_Consulta.xls')

In [464]:
df.head()

,Lugar,Espacialidade,COD IBGE,IDHM Renda (2010),IDHM Longevidade (2010),IDHM Educação (2010),Taxa de analfabetismo - 11 a 14 anos (2010),Taxa de analfabetismo - 15 a 17 anos (2010),Taxa de analfabetismo - 18 a 24 anos (2010),Taxa de analfabetismo - 25 a 29 anos (2010),Taxa de analfabetismo - 25 anos ou mais (2010),Taxa de analfabetismo - 15 anos ou mais (2010),Taxa de analfabetismo - 18 anos ou mais (2010),Renda per capita (2010)
0,Brasil,País,--,0.739,0.816,0.637,3.24,2.20,2.61,3.96,11.82,9.61,10.19,793.87
1,Aeroviários / Esplanada Anicuns / São José / A...,UDH,--,0.766,0.878,0.736,0.74,0.25,0.28,0.42,3.10,2.44,2.56,942.98
2,Água Branca / Riviera / Sonho Verde / Califórn...,UDH,--,0.786,0.908,0.775,0.78,0.64,0.60,0.59,3.46,2.81,2.95,1063.54
3,"Aldeia do Vale / Monte Verde (Goiânia , RM Goi...",UDH,--,1.000,0.940,0.920,0.00,0.00,0.88,0.00,0.33,0.38,0.40,6361.16
4,Alphaville Flamboyant / Jardins Munique / Port...,UDH,--,1.000,0.940,0.920,0.27,0.70,0.32,0.26,0.45,0.44,0.43,6361.16


In [465]:
df = df.drop(df[df['Espacialidade']!= 'UDH'].index)

In [466]:
dfFinal = pd.DataFrame(columns=df.columns)
dfFinal

,Lugar,Espacialidade,COD IBGE,IDHM Renda (2010),IDHM Longevidade (2010),IDHM Educação (2010),Taxa de analfabetismo - 11 a 14 anos (2010),Taxa de analfabetismo - 15 a 17 anos (2010),Taxa de analfabetismo - 18 a 24 anos (2010),Taxa de analfabetismo - 25 a 29 anos (2010),Taxa de analfabetismo - 25 anos ou mais (2010),Taxa de analfabetismo - 15 anos ou mais (2010),Taxa de analfabetismo - 18 anos ou mais (2010),Renda per capita (2010)


In [467]:
# este codigo serve para pegar mais de um bairro que tem mais de um complemento na mesma linha     
dfFinal = pd.DataFrame(columns=df.columns)
dfFinal['Complemento'] = np.NaN
for i, row in df.iterrows():
     #row['Id_linha_original'] = i+2 # criei este numero para ter um controle de qual linha originou os novos registros
                                    # esse + 2 é por conta da linha de cabeçalho no .xls e a contagem iniciar do 0
       
    # retirando (Goiania, RM Goiania) 
     ii = row['Lugar'].find('(Goiânia , RM Goiânia)')
     lugar  = row['Lugar']
     if ii != -1:
         lugar = lugar[0:ii]
            
     # existem vários padroes no arquivo, mas em grande parte está separado por barras /
     # alguns possuem um ou mais dois pontos':'
     # então a estratégia é pegar o que vem antes do ':' e assumir como bairro, e o seguinte como complemento
     # para os complementos que possuem /, serão divididos em novas linhas, uma linha pra cada, invés de deixar agrupado
     # quando houver mais que um ':' , pegar o texto entre a barra anterior e os dois pontos como bairro, criando nova linha
    
    
     #criando o array de bairros 
     bairros= []
     if lugar.count(":")>=1 :
         import re
         
         indexponto = [m.start() for m in re.finditer(':', lugar)] # esta linha pega o index de todas as ocorrencias de ':'
        
         
        
         i = 0 #era pra ser um for, mas o python n faz o for classico
         while i < len(indexponto):
            bairros =[]
            #pega o nome do bairro
            nomeBairro = lugar[0:indexponto[i]] # por padrao pega do inicio ao ':'
            if i !=0 and nomeBairro.rfind("/")>0:  # se não for a primeira iteracao e houver barra
                nomeBairro = lugar[nomeBairro.rfind("/")+1:indexponto[i]] # o nome do bairro sera da barra até os pontos
           
            #pega o complemento
            complementoBruto = lugar[indexponto[i]+1:len(lugar)]# este +1 é pra pular os dois pontos
            if i+1<len(indexponto)  : 
                complementoBruto = lugar[indexponto[i]+1:indexponto[i+1]]
                complementoBruto = complementoBruto[0:complementoBruto.rfind("/")]
                
            
            for comp in complementoBruto.split("/") : 
                bairros.append(nomeBairro +" ( " + comp +" )") # formata para o a forma padrao de processamento
                
            i = i+1    
                
     
         
     else:
         bairros = lugar.split("/") # processo padrão, na maioria das aglomerações, os bairros sao separadas por barras
         
        #reescrive tudo em um novo DataFrame, processando todos da mesma forma
     for bairro in bairros:        
           complemento = np.NaN
           ii = bairro.find('(')
           if ii != -1 :
                bairro = bairro[:-2]
                    
           if ii != -1 :
                complemento = bairro[ii+1:len(bairro)].strip()
                bairro = bairro[0:ii]
                
           
           rowTeste = row        
           rowTeste['Complemento'] = complemento
           rowTeste['Lugar'] = bairro.strip()
           dfFinal.loc[dfFinal.size] = rowTeste
print("primeira iteracao")

primeira iteracao


In [468]:
for i, row in dfFinal.iterrows():
     if row['Lugar'].find('Unidade Territorial de Planejamento')==0 :
        dfFinal.set_value(index=i,col='Lugar',value=row['Lugar'][36:])
       

In [469]:
dfFinal.dtypes

Lugar                                              object
Espacialidade                                      object
COD IBGE                                           object
IDHM Renda (2010)                                 float64
IDHM Longevidade (2010)                           float64
IDHM Educação (2010)                              float64
Taxa de analfabetismo - 11 a 14 anos (2010)       float64
Taxa de analfabetismo - 15 a 17 anos (2010)       float64
Taxa de analfabetismo - 18 a 24 anos (2010)       float64
Taxa de analfabetismo - 25 a 29 anos (2010)       float64
Taxa de analfabetismo - 25 anos ou mais (2010)    float64
Taxa de analfabetismo - 15 anos ou mais (2010)    float64
Taxa de analfabetismo - 18 anos ou mais (2010)    float64
Renda per capita (2010)                           float64
Complemento                                        object
dtype: object

In [470]:
dfFinal.sort_values(by=['Lugar'],inplace=True)
dfFinal['Lugar'].value_counts(dropna=False)
dfFinal.head(50)
dfLugar = dfFinal['Lugar']

dfFinal['contagem'] = 1

dfFinal =dfFinal.groupby('Lugar', sort=False).sum()
dfFinal.reset_index(inplace=True)
dfFinal.dtypes

Lugar                                              object
IDHM Renda (2010)                                 float64
IDHM Longevidade (2010)                           float64
IDHM Educação (2010)                              float64
Taxa de analfabetismo - 11 a 14 anos (2010)       float64
Taxa de analfabetismo - 15 a 17 anos (2010)       float64
Taxa de analfabetismo - 18 a 24 anos (2010)       float64
Taxa de analfabetismo - 25 a 29 anos (2010)       float64
Taxa de analfabetismo - 25 anos ou mais (2010)    float64
Taxa de analfabetismo - 15 anos ou mais (2010)    float64
Taxa de analfabetismo - 18 anos ou mais (2010)    float64
Renda per capita (2010)                           float64
contagem                                            int64
dtype: object

In [471]:
dfF = pd.DataFrame(columns=dfFinal.columns)
for  i, row in dfFinal.iterrows():
   
    row['IDHM Renda (2010)']  = row['IDHM Renda (2010)']  / row['contagem']
    row['IDHM Longevidade (2010)']  = row['IDHM Longevidade (2010)']  / row['contagem']
    row['IDHM Educação (2010)']  = row['IDHM Educação (2010)']  / row['contagem']
    row['Taxa de analfabetismo - 11 a 14 anos (2010)']  = row['Taxa de analfabetismo - 11 a 14 anos (2010)']  / row['contagem']
    row['Taxa de analfabetismo - 15 a 17 anos (2010)']  = row['Taxa de analfabetismo - 15 a 17 anos (2010)']  / row['contagem']
    row['Taxa de analfabetismo - 18 a 24 anos (2010)']  = row['Taxa de analfabetismo - 18 a 24 anos (2010)']  / row['contagem']
    row['Taxa de analfabetismo - 25 a 29 anos (2010)']  = row['Taxa de analfabetismo - 25 a 29 anos (2010)']  / row['contagem']
    row['Taxa de analfabetismo - 25 anos ou mais (2010)']  = row['Taxa de analfabetismo - 25 anos ou mais (2010)']  / row['contagem']
    row['Taxa de analfabetismo - 15 anos ou mais (2010)']  = row['Taxa de analfabetismo - 15 anos ou mais (2010)']  / row['contagem']
    row['Taxa de analfabetismo - 18 anos ou mais (2010)'] = row['Taxa de analfabetismo - 18 anos ou mais (2010)'] /  row['contagem']
    row['Renda per capita (2010)'] = row['Renda per capita (2010)'] /   row['contagem']
    dfF.loc[dfF.size] = row

In [472]:
del dfF['contagem']
dfF.head(50)

,Lugar,IDHM Renda (2010),IDHM Longevidade (2010),IDHM Educação (2010),Taxa de analfabetismo - 11 a 14 anos (2010),Taxa de analfabetismo - 15 a 17 anos (2010),Taxa de analfabetismo - 18 a 24 anos (2010),Taxa de analfabetismo - 25 a 29 anos (2010),Taxa de analfabetismo - 25 anos ou mais (2010),Taxa de analfabetismo - 15 anos ou mais (2010),Taxa de analfabetismo - 18 anos ou mais (2010),Renda per capita (2010)
0,Abajá,0.753000,0.850000,0.732500,1.750000,0.420000,0.475000,0.755000,4.200000,3.340000,3.525000,874.035000
13,Aeroviários,0.766000,0.878000,0.736000,0.740000,0.250000,0.280000,0.420000,3.100000,2.440000,2.560000,942.980000
26,Aldeia do Vale,1.000000,0.940000,0.920000,0.000000,0.000000,0.880000,0.000000,0.330000,0.380000,0.400000,6361.160000
39,Alpes,0.833000,0.905000,0.847000,0.730000,0.380000,0.320000,0.390000,1.910000,1.570000,1.640000,1425.060000
52,Alphaville Flamboyant,1.000000,0.940000,0.920000,0.270000,0.700000,0.320000,0.260000,0.450000,0.440000,0.430000,6361.160000
65,Alto da Glória,0.987000,0.913000,0.913000,0.000000,0.000000,0.130000,0.160000,0.450000,0.370000,0.380000,3714.740000
78,Alvorada,0.759000,0.871000,0.762000,1.140000,0.580000,1.200000,0.850000,3.910000,3.260000,3.420000,899.620000
91,Ana Flávia,0.720000,0.846000,0.705000,2.300000,0.780000,0.970000,0.900000,5.630000,4.530000,4.810000,706.580000
104,"Aruanã I, II e III",0.799000,0.893000,0.786000,0.240000,0.850000,0.740000,0.560000,2.500000,2.130000,2.200000,1157.180000
117,Aurora,0.771000,0.884000,0.776000,0.720000,0.150000,0.500000,0.400000,3.110000,2.480000,2.610000,969.310000


In [473]:
dfF.to_excel('AtlasBairrosMedia.xls')

In [474]:
print("gravou")


gravou
